In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import requests


# Loading Frames

In [ ]:
url = 'https://nist-assets.s3.us-east-1.amazonaws.com/shake.mp4'
os.makedirs('tmp', exist_ok=True)
with open('tmp/shake.mp4', 'wb') as f:
    f.write(requests.get(url).content)


In [ ]:
cap = cv2.VideoCapture('tmp/shake.mp4')
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
fps = cap.get(cv2.CAP_PROP_FPS)

# reference frames evenly spaced across the video
num_ref_frames = 10
frame_indices = [int(i * frame_count / num_ref_frames) for i in range(num_ref_frames)]

# Store all frames in an array
all_frames = []
selected_frames = []
current_idx = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break
    rotated = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    frame_rgb = cv2.cvtColor(rotated, cv2.COLOR_BGR2RGB)
    all_frames.append(frame_rgb)
    if current_idx in frame_indices:
        selected_frames.append(frame_rgb)
    current_idx += 1
cap.release()

# Display reference frames with frame number and timestamp
fig, axes = plt.subplots(1, num_ref_frames, figsize=(20, 4))
for i, frame in enumerate(selected_frames):
    frame_number = frame_indices[i]
    time_sec = frame_number / fps
    axes[i].imshow(frame)
    axes[i].axis('off')
    axes[i].set_title(f'Frame {frame_number}\n{time_sec:.2f}s')
fig.tight_layout()
fig.show()


In [ ]:
guide_frame_idx = 0
guide_frame = all_frames[guide_frame_idx]

fig, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.imshow(guide_frame)
ax.axis('off')
ax.set_title(f'Frame {guide_frame_idx}')
fig.show()

frame_rgb = guide_frame.copy()
frame_hsv = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2HSV)

# Hue Range

In [ ]:
def show_hue_range(start_hue: int, end_hue: int, s_min=100, s_max=255, v_min=100, v_max=255, axes=None):
    # horizontal HSV gradient
    height, width = 100, 180
    h = np.linspace(0, 179, width).astype(np.uint8)
    s = np.full((height,), s_max, dtype=np.uint8)
    v = np.full((height,), v_max, dtype=np.uint8)

    hsv_img = np.zeros((height, width, 3), dtype=np.uint8)
    for i in range(width):
        hsv_img[:, i, 0] = h[i]
        hsv_img[:, i, 1] = s
        hsv_img[:, i, 2] = v

    # Handle wrapping
    if start_hue < end_hue:
        lower = np.array([start_hue, s_min, v_min])
        upper = np.array([end_hue, s_max, v_max])
        mask = cv2.inRange(hsv_img, lower, upper)
    else:
        lower1 = np.array([start_hue, s_min, v_min])
        upper1 = np.array([179, s_max, v_max])
        lower2 = np.array([0, s_min, v_min])
        upper2 = np.array([end_hue, s_max, v_max])
        mask1 = cv2.inRange(hsv_img, lower1, upper1)
        mask2 = cv2.inRange(hsv_img, lower2, upper2)
        mask = cv2.bitwise_or(mask1, mask2)

    rgb_img = cv2.cvtColor(hsv_img, cv2.COLOR_HSV2RGB)
    result = cv2.bitwise_and(rgb_img, rgb_img, mask=mask)

    axes[0].imshow(rgb_img)
    axes[1].imshow(mask, cmap='gray')
    axes[2].imshow(result)
    for a in axes:
        a.set_yticks([])
        a.set_xticks([0, 45, 90, 135, 180])
    
    if start_hue < end_hue:
        return [[lower, upper]]
    else:
        return [[lower1, upper1], [lower2, upper2]]

fig, axes = plt.subplots(3, 1, figsize=(15, 4), sharex=True)

start_hue = 145
hue_length = 45
s_min, v_min = 100, 100
s_max, v_max = 255, 255
end_hue = (start_hue + hue_length) % 180

hue_range = show_hue_range(start_hue=start_hue, end_hue=end_hue, s_min=s_min, s_max=s_max, v_min=v_min, v_max=v_max, axes=axes)
fig.tight_layout()

hue_range

In [ ]:
mask1 = cv2.inRange(frame_hsv, hue_range[0][0], hue_range[0][1])
mask2 = cv2.inRange(frame_hsv, hue_range[1][0], hue_range[1][1])
red_mask = cv2.bitwise_or(mask1, mask2)

frame_masked = cv2.bitwise_and(frame_rgb, frame_rgb, mask=red_mask)

fig, axes = plt.subplots(1, 2, figsize=(20, 20))
axes[0].imshow(red_mask, cmap='gray')
axes[0].axis('off')
axes[1].imshow(frame_masked)
axes[1].axis('off')
fig.tight_layout()
fig.show()

# Edge Detection

In [ ]:
gauss_ksize = (101, 101)
gauss_sigmaX = 0
blurred = cv2.GaussianBlur(red_mask, ksize=gauss_ksize, sigmaX=gauss_sigmaX)

fig, axes = plt.subplots(1, 2, figsize=(20, 20))
axes[0].imshow(red_mask, cmap='gray')
axes[0].axis('off')
axes[1].imshow(blurred, cmap='gray')
axes[1].axis('off')
fig.tight_layout()

In [ ]:
# # Define threshold values to try
# threshold1_values = np.linspace(5, 40, 5).astype(int)
# threshold2_values = np.linspace(30, 60, 5).astype(int)

# fig, axes = plt.subplots(len(threshold1_values), len(threshold2_values), figsize=(30, 50))

# for i, t1 in enumerate(threshold1_values):
#     for j, t2 in enumerate(threshold2_values):
#         edges = cv2.Canny(blurred, threshold1=t1, threshold2=t2)
#         ax = axes[i, j]
#         ax.imshow(edges, cmap='gray')
#         ax.set_title(f"t1={t1}, t2={t2}")
#         ax.axis('off')

# plt.tight_layout()
# plt.show()


In [ ]:
# Canny Edge Detection
Canny_threshold1 = 5
Canny_threshold2 = 30
edges = cv2.Canny(blurred, threshold1=Canny_threshold1, threshold2=Canny_threshold2)

fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(edges, cmap='gray')
ax.set_title("Canny Edges")
ax.axis('off')
fig.show()

# Line Detection

In [ ]:
# Detect Lines with Hough Transform
rho = 1
theta = np.pi / 180
threshold = 30
min_line_length = 500
max_line_gap = 50
lines = cv2.HoughLinesP(edges, rho=rho, theta=theta, 
                        threshold=threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)

# Plot detected line segments
frame_lines_overlay = frame_rgb.copy()
if lines is not None:
    for line in lines:
        x1, y1, x2, y2 = line[0]
        cv2.line(frame_lines_overlay, (x1, y1), (x2, y2), (0, 255, 0), 2)

fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(frame_lines_overlay)
ax.set_title("Raw Hough Lines")
ax.axis('off')
fig.show()


# Line Fitting

In [ ]:
# Classify lines into horizontal and vertical
def separate_lines(lines):
    horiz, vert = [], []
    for line in lines:
        x1, y1, x2, y2 = line[0]
        angle = np.arctan2((y2 - y1), (x2 - x1)) * 180 / np.pi
        if -20 < angle < 20:
            horiz.append((x1, y1, x2, y2))
        elif 70 < abs(angle) < 110:
            vert.append((x1, y1, x2, y2))
    return horiz, vert

horiz_lines, vert_lines = separate_lines(lines) if lines is not None else ([], [])
print(f"Horizontal lines: {len(horiz_lines)}, Vertical lines: {len(vert_lines)}")


In [ ]:
# Fit single line to endpoints of detected lines
def fit_line(lines):
    if not lines:
        return None
    points = np.array([[[x1, y1]] for x1, y1, _, _ in lines] + [[[x2, y2]] for _, _, x2, y2 in lines])
    vx, vy, x0, y0 = cv2.fitLine(points, distType=cv2.DIST_L2, param=0, reps=0.01, aeps=0.01)
    return vx[0], vy[0], x0[0], y0[0]

fitted_h = fit_line(horiz_lines)
fitted_v = fit_line(vert_lines)

print("Fitted horizontal line params:", fitted_h)
print("Fitted vertical line params:", fitted_v)

# Plot fitted lines    
def draw_fitted_line(out_img, line_params, color, length=4000, thickness=8):
    if line_params is None:
        return
    vx, vy, x0, y0 = line_params
    pt1 = (int(x0 - length * vx), int(y0 - length * vy))
    pt2 = (int(x0 + length * vx), int(y0 + length * vy))
    cv2.line(out_img, pt1, pt2, color, thickness)

# Copy the original frame to draw on
fitted_img = frame_rgb.copy()

# Draw horizontal and vertical fitted lines
draw_fitted_line(fitted_img, fitted_h, (0, 255, 0))
draw_fitted_line(fitted_img, fitted_v, (0, 255, 0))

# Display
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(fitted_img)
ax.axis('off')
ax.set_title('Fitted Horizontal (Green) and Vertical (Blue) Lines')
fig.show()


# Intersection

In [ ]:
# Compute the intersection point of two lines
def compute_intersection(v1, v2):
    vx1, vy1, x1, y1 = v1
    vx2, vy2, x2, y2 = v2
    A = np.array([[vx1, -vx2], [vy1, -vy2]])
    b = np.array([x2 - x1, y2 - y1])
    try:
        t, s = np.linalg.solve(A, b)
        ix = x1 + t * vx1
        iy = y1 + t * vy1
        return int(ix), int(iy)
    except np.linalg.LinAlgError:
        return None

intersection = compute_intersection(fitted_h, fitted_v) if (fitted_h and fitted_v) else None

out_img = fitted_img.copy()

# Draw intersection dot
if intersection:
    cv2.circle(out_img, intersection, 15, (0, 0, 255), -1)

fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(out_img)
ax.set_title("Fitted Lines and Intersection (Thicker)")
ax.axis('off')
fig.show()


# Video

In [ ]:
# Determine frame size from one frame
h, w, _ = all_frames[0].shape

# Create a VideoWriter that outputs an MP4
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out_vid = cv2.VideoWriter(
    'tmp/output.mp4',
    fourcc,
    fps,
    (w, h)
)

intersection_points = []
red_pixel_percentages = []
skipped_frames = []
failing_frames = []

for i, frame_rgb in enumerate(all_frames):
    # Convert back to BGR for HSV conversion
    frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
    frame_hsv = cv2.cvtColor(frame_bgr, cv2.COLOR_BGR2HSV)

    # Build the red mask
    masks = [cv2.inRange(frame_hsv, lower, upper) for (lower, upper) in hue_range]
    red_mask = cv2.bitwise_or(*masks)

    # Calculate red pixel percentage
    red_pixels = cv2.countNonZero(red_mask)
    total_pixels = red_mask.shape[0] * red_mask.shape[1]
    red_percentage = (red_pixels / total_pixels) * 100
    red_pixel_percentages.append(red_percentage)

    # Skip if red mask is mostly empty
    if red_percentage < 1.0:
        intersection_points.append(None)
        skipped_frames.append(i)
        out_vid.write(frame_bgr)
        continue

    # Blur and edge detect
    blurred = cv2.GaussianBlur(red_mask, ksize=gauss_ksize, sigmaX=gauss_sigmaX)
    edges = cv2.Canny(blurred, threshold1=Canny_threshold1, threshold2=Canny_threshold2)

    # Hough lines
    lines = cv2.HoughLinesP(edges, rho=rho, theta=theta,
                            threshold=threshold, minLineLength=min_line_length, maxLineGap=max_line_gap)

    # Separate and fit
    if lines is not None:
        horiz_lines, vert_lines = separate_lines(lines)
        fitted_h = fit_line(horiz_lines)
        fitted_v = fit_line(vert_lines)
    else:
        fitted_h, fitted_v = None, None

    # Compute intersection
    if fitted_h is not None and fitted_v is not None:
        intersection = compute_intersection(fitted_h, fitted_v)
    else:
        intersection = None
        failing_frames.append(i)

    intersection_points.append(intersection)
    
    # Draw on a copy of the original
    annotated_rgb = frame_rgb.copy()

    # Draw fitted lines
    draw_fitted_line(annotated_rgb, fitted_h, color=(0, 255, 0))
    draw_fitted_line(annotated_rgb, fitted_v, color=(0, 255, 0))

    # Draw the intersection as a red circle
    if intersection is not None:
        cv2.circle(annotated_rgb, intersection, radius=15, color=(0, 0, 255), thickness=-1)

    annotated_bgr = cv2.cvtColor(annotated_rgb, cv2.COLOR_RGB2BGR)
    out_vid.write(annotated_bgr)
out_vid.release()

# print(f"Red pixel percentages per frame: {red_pixel_percentages}")
print(f"Computed intersection for {len(intersection_points)} frames.")
# print(f"Skipped frames (low red): {skipped_frames}")
print(f"Number of skipped frames: {len(skipped_frames)}")
print(f"Failing frames (enough red, but no intersection): {failing_frames}")
print(f"Number of failing frames: {len(failing_frames)}")


In [ ]:
num_failures = len(failing_frames)
if num_failures > 0:
    max_cols = 4
    cols = min(max_cols, num_failures)
    rows = int(np.ceil(num_failures / cols))

    fig, axes = plt.subplots(rows, cols, figsize=(5 * cols, 10 * rows))

    # Flatten axes and handle the case where it's not a 2D array
    axes = axes.flatten() if num_failures > 1 else [axes]

    for i, idx in enumerate(failing_frames):
        ax = axes[i]
        ax.imshow(all_frames[idx])
        ax.set_title(f"Frame {idx}")
        ax.axis('off')

    # Hide any unused subplots
    for j in range(num_failures, len(axes)):
        axes[j].axis('off')

    fig.tight_layout()
    fig.show()
else:
    print("All frames have intersections.")


In [ ]:
# Extract valid points and their frame indices
indices = [i for i, pt in enumerate(intersection_points) if pt is not None]
x_vals = [intersection_points[i][0] for i in indices]
y_vals = [intersection_points[i][1] for i in indices]

# Plot last frame as background and overlay points and connecting lines
fig, ax = plt.subplots(figsize=(20, 20))
ax.imshow(all_frames[-1])

# Plot intersection points
ax.scatter(x_vals, y_vals, c='red')

# Connect adjacent frames' points
for i in range(len(indices) - 1):
    idx1, idx2 = indices[i], indices[i + 1]
    if idx2 == idx1 + 1:  # Only connect if frames are consecutive
        x1, y1 = intersection_points[idx1]
        x2, y2 = intersection_points[idx2]
        ax.plot([x1, x2], [y1, y2], c='blue')

ax.set_title('Intersection Points Over Last Frame')
ax.axis('off')
fig.show()